In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

In [ ]:
!python --version

In [ ]:
!pip install datasets

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import torch
import pickle
from tqdm import tqdm
from torch.nn.functional import cosine_similarity
from sklearn.metrics import ndcg_score
from matplotlib import pyplot as plt

drive_root = '/content/drive/MyDrive/Colab/vp'

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
from datasets import load_dataset, list_datasets
# BeIR/trec-covid
# BeIR/trec-covid-qrels
# BeIR/trec-covid-generated-queries
# BeIR/trec-news-generated-queries
corpus_dataset = load_dataset("BeIR/trec-covid", "corpus")['corpus']
queries_dataset = load_dataset("BeIR/trec-covid", "queries")['queries']
qrels_dataset = load_dataset("BeIR/trec-covid-qrels")['test']
corpus_dataset, queries_dataset, qrels_dataset

In [ ]:

!pip install git+https://github.com/UKPLab/sentence-transformers.git

In [ ]:
# !pip uninstall sentence-transformers

# MiniLM inferense



In [ ]:
# !pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Muennighoff/SGPT-1.3B-weightedmean-nli-bitfit")  # This one requires latest installation of sentence-transformers straight from the repo
# model = SentenceTransformer('sentence-transformers/all-roberta-large-v1')
# model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# from google.colab import drive
# import pickle
# drive.mount('/content/drive')
# drive_root = '/content/drive/MyDrive/Colab/vp'

### Embedding calculation



In [ ]:
experiment_name = 'SGPT-1.3B-weightedmean-nli-bitfit'
# 1,2 - all
# 3 ms-macro | misuse | bs
# def encode_batch(batch_start: int, batch_end: int):
#     subsection_of_dataset = corpus_dataset[batch_start:batch_end]
#     sentences = [f"{title}. {text}" for title, text in zip(subsection_of_dataset['title'], subsection_of_dataset['text'])]
#     sentence_embeddings = model.encode(sentences, show_progress_bar=True)
#     np.save(f'{drive_root}/{experiment_name}_embeddings_{batch_start}:{batch_end}.npy', sentence_embeddings)
def encode_queries_batch(batch_start: int, batch_end: int):
    subsection_of_dataset = queries_dataset[batch_start:batch_end]
    query_embeddings = model.encode(subsection_of_dataset['text'], show_progress_bar=True)
    np.save(f'{drive_root}/{experiment_name}_query_embeddings_{batch_start}:{batch_end}.npy', query_embeddings)   

def encode_documents_with_openai_encoded_ids():
    corpus_id_to_index_mapping = {_id: i for i, _id in enumerate(corpus_dataset['_id'])}
    sentences = [f"{corpus_dataset[corpus_id_to_index_mapping[_id]]['title']}. {corpus_dataset[corpus_id_to_index_mapping[_id]]['text']}" for _id in np.load(f'{drive_root}/openai/encoded_ids.npy')]
    sentence_embeddings = model.encode(sentences, show_progress_bar=True)
    np.save(f'{drive_root}/{experiment_name}_embeddings_from_openai_encoded_ids.npy', sentence_embeddings)

In [ ]:
%%time
encode_documents_with_openai_encoded_ids()

In [ ]:
# Time of encoding 1000 first sentences with all-MiniLM-L6-v2
# CPU (free colab):  
# CPU times: user 2min 7s, sys: 365 ms, total: 2min 7s
# Wall time: 2min 9s
# GPU (free colab):
# CPU times: user 5.57 s, sys: 1.26 s, total: 6.83 s
# Wall time: 14.3 s


In [ ]:
encode_queries_batch(0, None)

### Similiarity computation

In [ ]:
# def to_tensor(embeddings_dict):
#     return {_id: torch.tensor(embedding, device=device) for _id, embedding in embeddings_dict.items()}

experiment_name = 2
docs_ids = corpus_dataset['_id']
docs_embeddings = torch.tensor(np.load(f"{drive_root}/{experiment_name}_embeddings_0:None.npy"), device=device)
query_ids = queries_dataset['_id']
query_embeddings = torch.tensor(np.load(f"{drive_root}/{experiment_name}_query_embeddings_0:None.npy"), device=device)

In [ ]:
#  query_embeddings.shape

In [ ]:
def true_scores(query_id):
    pd_qrels = pd.DataFrame(qrels_dataset)    
    pd_relevant_qrels = pd_qrels[pd_qrels['query-id'] == query_id]
    qrels = dict(zip(pd_relevant_qrels['corpus-id'], pd_relevant_qrels['score']))
    return [qrels[doc_id] if doc_id in qrels else 0 for doc_id in docs_ids]


In [ ]:
# ndcg_scores = []
# for i, query_id in enumerate(query_ids):
#     cos_sims = cosine_similarity(query_embeddings[i], docs_embeddings).cpu().numpy()
#     _ndcg_score = ndcg_score([true_scores(int(query_id))], [cos_sims], k=10) 
#     ndcg_scores.append(_ndcg_score)
#     print(_ndcg_score)
ndcg_scores = [0.1100458831490401, 0.49276192198755825, 0.15362568563894313, 0.38431135260440824, 0.2899773651602119, 0.17010317160287547, 0.4085359184257957, 0.06625422345438896, 0.0, 0.13698471029831177, 0.3995688713838976, 0.46467597166437496, 0.39366982114002336, 0.4373524791503101, 0.2914893383161489, 0.8582425318466904, 0.496314133361996, 0.5708588662113935, 0.4826073387355251, 0.7799082337019198, 0.25880978898961277, 0.4440973278132557, 0.5698897789767065, 0.31093196555772146, 0.0, 0.23758350840568815, 0.8275534295258489, 0.5912463455842708, 0.6515402749735272, 0.9216017310213245, 0.4511074772466062, 0.0, 0.6203974344166848, 0.0, 0.03312711172719448, 0.6906478832608419, 0.4641183337813128, 0.36483361535360737, 0.9633180389503199, 0.9574284411791895, 0.9681896059005243, 0.8899541168509599, 0.8701249883466594, 0.7818745837832112, 0.6535073651712326, 0.866947989864271, 0.37307760663528433, 0.94497705842548, 0.333391619621485, 0.4225749983705864]
# np.average(ndcg_scores) = 0.4770028047513443

In [ ]:

plt.rcParams["figure.figsize"] = (14, 20)
PROPS = {
    'boxprops':{'edgecolor':'black', 'zorder': 2},
    'medianprops':{'color':'black'},
    'whiskerprops':{'color':'black'},
    'capprops':{'color':'black'}
}

def generate_dists(_query_ids):

    plt.figure()
    labels = []
    cos_sims = []
    for i, query_id in enumerate(_query_ids):
        current_cos_sims = cosine_similarity(query_embeddings[int(query_id) - 1], docs_embeddings).cpu().numpy()
        cos_sims.extend(current_cos_sims)
        labels.extend([f"id = {query_id}; NDCG@10 = {ndcg_scores[int(query_id) - 1]:.2f}" for i in range(len(current_cos_sims))])

    print()
    df=pd.DataFrame({'queries': labels, 'cos_sims': cos_sims})        
    sns.violinplot(data=df, x='cos_sims', y='queries', inner=None, color='#5FBEF2')
    sns.boxplot(data=df, x='cos_sims', y='queries', saturation=0.5, width=0.2, color='#CCF6FF', **PROPS)
    # plt.legend()   
    plt.grid(axis='x')
    plt.savefig(f"{drive_root}/violin_fig_{_query_ids[0]}-{_query_ids[-1]}.png", dpi=300,  bbox_inches='tight')
    plt.show()
generate_dists(query_ids[0:10])

In [ ]:
for i in range(5):
    generate_dists(query_ids[10*i:10*(i+1)]) 

## OpenAI API:


Estimate:
We have money for ~ 100 Mtokens (Mega tokens, aka 10^6 tokens)



In [ ]:
import string
word_counts = []
for text in tqdm(corpus_dataset['text']):
    word_counts.append(len(text.translate(str.maketrans('', '', string.punctuation)).split()))

In [ ]:
word_counts_df = pd.DataFrame(word_counts)
word_counts_df.describe()

In [ ]:

# sns.scatterplot(pd.DataFrame(word_counts))
# sns.histplot(x=word_counts_df[0])

148 words per document * 1.33 tokens per word ~= 200 tokens/document

Which gives us 500 000 documents. Which is almost 3 times the amount we need.

In [ ]:
!pip install openai

In [ ]:
import openai
from getpass import getpass
openai.api_key = getpass(prompt="OpenAI API key: ")

In [ ]:
# response = openai.Embedding.create(input = ["Build advanced search, clustering, topic modeling, and classification functionality with our embeddings offering.",
#                                             "Second sentense in a batch"],
#                                    model="text-embedding-ada-002")

In [ ]:
with open(f"{drive_root}/model_esponse.pkl", "rb") as file:
    model_response = pickle.load(file)

In [ ]:
np.array(model_response['data'][0]['embedding'])[None,:]


In [ ]:
response["usage"]

In [ ]:
from time import sleep
import os
def query_openai(query: str, max_try: int = 3, **kwargs):
    for _ in range(max_try):
        try:
            response = openai.Embedding.create(input=query, model="text-embedding-ada-002")
            return response['data'][0]['embedding'], response["usage"]["total_tokens"]
            raise ValueError(f'Invalid request type: {req_type}')

        except Exception as e:
            print("Error in request: ", e)
            sleep(3)

    return None, None

usd_per_token = 0.0004/1000
# global_token_usage = 12087329 # And we missed this count for about 1000 documets in the beggining
with open(f"{drive_root}/global_token_usage.pkl", "rb") as file:
    global_token_usage = pickle.load(file)
print(f"Tokens spent: {global_token_usage}, Money spent: {global_token_usage*usd_per_token:.2f}$")

In [ ]:
from datetime import datetime

too_big_ids = []#["ij3ncdb6", "gvh0wdxn", "c4pt07zk", "1vimqhdp", "pd1g119c"]


def choose_not_encoded_ids(number: int, _embeddings_df) -> list:
    already_mapped_ids = set(_embeddings_df['_id'])
    total_ids_pool = set(corpus_dataset['_id'])
    return list(np.random.choice(list(total_ids_pool - already_mapped_ids), size=number, replace=False))


def openai_encode_batch(number: int):
    error_count = 0
    if os.path.exists(f'{drive_root}/openai/'):

        global global_token_usage
        with open(f"{drive_root}/global_token_usage.pkl", "rb") as file:
            global_token_usage = pickle.load(file)

        print("Loading existing embeddings")
        # _embeddings_df = pd.read_csv(f'{drive_root}/openai/text-embedding-ada-002_v2.csv')
        _embeddings_df = pd.read_pickle(f'{drive_root}/openai/text-embedding-ada-002_v2.pkl')
        # print("Creating backup")
        # _embeddings_df.to_csv(f'{drive_root}/openai/text-embedding-ada-002_v2.csv.backup.{datetime.now()}', index=False)
        print("Finished backup")

        document_ids = choose_not_encoded_ids(number, _embeddings_df)

        already_mapped_ids = set(_embeddings_df['_id'])

        total_token_usage = 0
        corpus_id_to_index_mapping = {_id: i for i, _id in enumerate(corpus_dataset['_id'])}
        _pd_embedding_column_names = [str(i) for i in range(1536)]
        _batch_df = pd.DataFrame(columns=_embeddings_df.columns)
        with tqdm(total=number, position=0, leave=True) as pbar:
            pbar.set_postfix_str(f"Saved {len(_embeddings_df['_id'])} embeddings")
            for i, _id in enumerate(document_ids):
                if _id not in already_mapped_ids:
                    if _id not in too_big_ids:
                        document_entry = corpus_dataset[corpus_id_to_index_mapping[_id]]
                        sentence = f"{document_entry['title']}. {document_entry['text']}"
                        embedding, tokens_used = query_openai(sentence)

                        

                        if embedding is not None:
                            total_token_usage += tokens_used
                            global_token_usage += tokens_used
                            new_row = pd.DataFrame(np.array(embedding)[None, :], columns=_pd_embedding_column_names)
                            new_row['_id'] = _id
                            _batch_df = pd.concat([_batch_df, new_row])
                            if (i - 501) % 1000 == 0:
                                _embeddings_df = pd.concat([_embeddings_df, _batch_df])
                                # _embeddings_df.to_csv(f'{drive_root}/openai/text-embedding-ada-002_v2.csv', index=False)
                                _embeddings_df.to_pickle(f'{drive_root}/openai/text-embedding-ada-002_v2.pkl')

                                _batch_df = pd.DataFrame(columns=_embeddings_df.columns)

                                pbar.set_postfix_str(f"Saved {len(_embeddings_df['_id'])} embeddings")

                                np.save(f'{drive_root}/openai/encoded_ids.npy', list(_embeddings_df['_id']))
                                with open(f"{drive_root}/global_token_usage.pkl", "wb") as file:
                                    pickle.dump(global_token_usage, file)
                            already_mapped_ids.add(_id)              
                        else:
                            print(f"Error!: embedding is None for id {_id}")
                            error_count += 1
                            if error_count > 10:
                                break
                    else:
                        [print(f"skipping id {_id} for being too fat")]    
                else:
                    print(f'id {_id} is already embedded. skipping...')
                pbar.update()
            

        _embeddings_df = pd.concat([_embeddings_df, _batch_df])
        # _embeddings_df.to_csv(f'{drive_root}/openai/text-embedding-ada-002_v2.csv', index=False)
        _embeddings_df.to_pickle(f'{drive_root}/openai/text-embedding-ada-002_v2.pkl')
        np.save(f'{drive_root}/openai/encoded_ids.npy', list(_embeddings_df['_id']))

        print(f"Token usage: {total_token_usage}, Global token usage: {global_token_usage}"),
        print(f"Money spent: {usd_per_token*total_token_usage}, Global money spent: {usd_per_token*global_token_usage}")
        print(f"Global documents encoded: {len(already_mapped_ids)}")
        with open(f"{drive_root}/global_token_usage.pkl", "wb") as file:
            pickle.dump(global_token_usage, file)
    else:
        raise Exception("No path. Drive not mounted?")


openai_encode_batch(5)
# This model's maximum context length is 8191 tokens however you requested 8510 tokens

In [ ]:
# embeddings_df = pd.read_csv(f'{drive_root}/openai/text-embedding-ada-002_v2.csv')
embeddings_df = pd.read_pickle(f'{drive_root}/openai/text-embedding-ada-002_v2.pkl')

In [ ]:
len(corpus_dataset['_id']) - 171332

In [ ]:

len(np.load(f'{drive_root}/openai/encoded_ids.npy'))

In [ ]:
# document_ids = ['ug7v899j', '9785vg6d']
# corpus_dataset[2001]
# pd.read_csv(f'{drive_root}/openai/text-embedding-ada-002_v2.csv')
# pd.read_csv(f'{drive_root}/openai/text-embedding-ada-002_v2.csv.backup.2023-01-26 20:09:33.390034')

In [ ]:
# def openai_encode_queries():
#     if os.path.exists(f'{drive_root}/openai/'):
#         total_token_usage = 0
#         for _id, query in tqdm(zip(queries_dataset["_id"], queries_dataset['text']), total=50):
#             filename = f'{drive_root}/openai/query-embedding-ada-002_v2_query_id_{_id}.npy'
#             if not os.path.exists(filename):
#                 embedding, tokens_used = query_openai(query)
#                 total_token_usage += tokens_used

#                 if embedding is not None:
#                     np.save(filename, embedding)
#                 else:
#                     raise Exception()
#             else:
#                 print(f'id {_id} is already embedded. skipping...')
#         global global_token_usage
#         global_token_usage += total_token_usage
#         print(f"Token usage: {total_token_usage}, Global token usage: {global_token_usage}"),
#         print(f"Money spent: {usd_per_token*total_token_usage}, Global money spent: {usd_per_token*global_token_usage}")
#     else:
#         raise Exception("No path. Drive not mounted?") 
# # openai_encode_queries()

Updated estimate: turns out, we have about 350 tokens per document which give us about 72k documents per token 10$

Samples of tokens spent per 200 documents:

- 200: 67324
- 200: 64815
- 200: 69195



In [ ]:
# embeddings_df.to_csv(f'{drive_root}/openai/text-embedding-ada-002_v2.csv', index=False)
embeddings_df = pd.read_pickle(f'{drive_root}/openai/text-embedding-ada-002_v2.pkl')
# query_embeddings_df.to_csv(f'{drive_root}/openai/query-embedding-ada-002_v2.csv', index=False)
query_embeddings_df = pd.read_csv(f'{drive_root}/openai/query-embedding-ada-002_v2.csv')


In [ ]:
def true_scores(query_id):
    pd_qrels = pd.DataFrame(qrels_dataset)    
    pd_relevant_qrels = pd_qrels[pd_qrels['query-id'] == int(query_id)]
    qrels = dict(zip(pd_relevant_qrels['corpus-id'], pd_relevant_qrels['score']))
    return [qrels[doc_id] if doc_id in qrels else 0 for doc_id in corpus_dataset['_id']]


def true_scores_on_subset(query_id, corpus_ids_ordered_list):
    pd_qrels = pd.DataFrame(qrels_dataset) 
    pd_relevant_qrels = pd_qrels[pd_qrels['query-id'] == int(query_id)]
    qrels = dict(zip(pd_relevant_qrels['corpus-id'], pd_relevant_qrels['score']))
    return [qrels[doc_id] if doc_id in qrels else 0 for doc_id in corpus_ids_ordered_list]


def calculate_ndcg_scores_for_openai():
    ndcg_scores = []
    documents_embeddings = torch.tensor(embeddings_df.drop('_id', axis=1).to_numpy(), device=device)
    for row, query_id in tqdm(zip(query_embeddings_df.drop('_id', axis=1).itertuples(index=False), query_embeddings_df['_id']), total=50):
        query_embedding = torch.tensor(row, device=device)
        cos_sims = cosine_similarity(query_embedding, documents_embeddings).cpu().numpy()
        ndcg_scores.append(ndcg_score([true_scores_on_subset(query_id, list(embeddings_df['_id']))], [cos_sims], k=10))
    return ndcg_scores

# 55%:
# openai_ndcg_scores = [0.5842585682254681, 1.0, 0.19218497482654184, 0.6442227883107535, 0.5437023030721884, 0.5009732673492617, 0.835780413693672, 0.5199967979955744, 0.2278403745052797, 0.6402900881481708, 0.17318663334822615, 0.3962756110857297, 0.5769766371786422, 0.6493026116472478, 0.7736502375634358, 0.9350624941733298, 0.8334430272969794, 0.4293039137177177, 0.48153499054313026, 0.8603818544462509, 0.3340514446642156, 0.9266360779006398, 0.7549357566339395, 0.7951657524814092, 0.9305687780632227, 0.5676818677290052, 0.7885497208855956, 0.5337225282363591, 0.8115558478722845, 0.9633180389503199, 0.2200917662980802, 0.12096982779948588, 0.2977473344546329, 0.3222722491219548, 0.46664232174566644, 0.8115558478722845, 0.4525070087200181, 0.4458221167281372, 0.8568270051237654, 0.9652843890316114, 0.9032521000738538, 0.9574284411791895, 1.0, 0.7407090992125821, 0.6168254041215527, 0.7389346265950472, 0.7123736502858729, 0.8178102956254404, 0.667905452944332, 0.5400096219094884]
# 100%:
openai_ndcg_scores = [0.8070980744903451, 1.0, 0.45484422125791807, 0.7207837995611228, 0.48268650276883, 0.5141863642577488, 1.0, 0.604685538711841, 0.1159354809201706, 0.7325140515171444, 0.40270657974248264, 0.19602998040580386, 0.5850494105837172, 0.6042846647453803, 0.8945466420562734, 0.8701249883466594, 0.9305687780632227, 0.5528346753887203, 0.4093911693176109, 1.0, 0.35687885840597894, 0.8276325935591538, 0.9266360779006398, 0.5661035610769689, 0.9305687780632227, 0.6036081270984908, 0.9652843890316114, 0.44821456061271253, 0.7763057112117956, 1.0, 0.5362234040097508, 0.19575819879232836, 0.17630010660342907, 0.473571703045486, 0.5685311061462373, 0.8643145546088338, 0.4209736685921866, 0.45403346208984086, 0.9305687780632227, 1.0, 0.9363792118010483, 1.0, 0.9574284411791895, 0.8244555950767655, 0.6941125739107519, 0.8945466420562734, 0.5384109300791363, 0.8801412713899903, 0.6882033310777197, 0.6769023376372896]
# openai_ndcg_scores = calculate_ndcg_scores_for_openai()


def calculate_ndcg_scores_sentence_transformer_full_dataset(experiment_name):
    docs_miniLM_embeddings = torch.tensor(np.load(f"{drive_root}/{experiment_name}_embeddings_0:None.npy"), device=device)
    query_miniLM_embeddings =  torch.tensor(np.load(f"{drive_root}/{experiment_name}_query_embeddings_0:None.npy"), device=device)

    query_ids = queries_dataset['_id']

    ndcg_scores = []
    for i, query_id in enumerate(tqdm(query_ids)):
        cos_sims = cosine_similarity(query_miniLM_embeddings[i], docs_miniLM_embeddings).cpu().numpy()
        _ndcg_score = ndcg_score([true_scores(query_id)], [cos_sims], k=10)
        ndcg_scores.append(_ndcg_score)

    return(ndcg_scores)

# def calculate_ndcg_scores_sentence_transformer_limited_to_openai(experiment_name):
#     corpus_ids_sublist =  np.load(f'{drive_root}/openai/encoded_ids.npy').tolist()
#     # docs_miniLM_embeddings_dict = {_id: embedding for embedding, _id in zip(np.load(f"{drive_root}/{experiment_name}_embeddings_from_openai_encoded_ids.npy"), np.load(f'{drive_root}/{experiment_name}_embeddings_from_openai_encoded_ids.npy')) 
#     #                       if _id in corpus_ids_subset}
#     docs_miniLM_embeddings_ordered_by_corpus_ids_sublist = \
#         torch.tensor(np.load(f"{drive_root}/{experiment_name}_embeddings_from_openai_encoded_ids.npy"), device=device)
#     query_miniLM_embeddings =  torch.tensor(np.load(f"{drive_root}/{experiment_name}_query_embeddings_0:None.npy"), device=device)

#     query_ids = queries_dataset['_id']

#     ndcg_scores = []
#     for i, query_id in enumerate(tqdm(query_ids, total=len(query_ids))):
#         cos_sims = cosine_similarity(query_miniLM_embeddings[i], docs_miniLM_embeddings_ordered_by_corpus_ids_sublist).cpu().numpy()
#         _ndcg_score = ndcg_score([true_scores_on_subset(query_id, corpus_ids_sublist)], [cos_sims], k=10)
#         ndcg_scores.append(_ndcg_score)

#     return(ndcg_scores)


# 55%:
# minilm_ndcg_scores_masked_to_openai = [0.14317299487623458, 0.3316936730892072, 0.05502294157452005, 0.2489083270225946, 0.31871739979946917, 0.2628934232775421, 0.2528410271851775, 0.15744006533381547, 0.0, 0.07839826897867538, 0.2200917662980802, 0.414764616039942, 0.2489083270225946, 0.45449827622138, 0.32291168898636957, 0.7787274229518825, 0.25480737726646885, 0.6780494608112012, 0.5503088515264319, 1.0, 0.3668893220573615, 0.32891754857327965, 0.4585942246815616, 0.2883028873610169, 0.0, 0.1100458831490401, 0.6332196796270029, 0.5993294818993699, 0.7212476289592209, 0.9652843890316114, 0.25677372734776027, 0.0, 0.4440973278132557, 0.0, 0.06943122193677731, 0.6563033086078547, 0.34184825634124033, 0.3706398689768581, 1.0, 0.8627260553676397, 0.8354093773782707, 0.8899541168509599, 0.793584067764911, 0.7460573260980901, 0.6061131829864573, 0.5541432109622958, 0.4035015715464804, 0.8572048559638626, 0.25983227167304046, 0.2934556883974403]
# minilm_ndcg_scores_masked_to_openai = calculate_ndcg_scores_sentence_transformer_limited_to_openai(2)
# 100%:
# minilm_ndcg_scores = calculate_ndcg_scores_sentence_transformer_full_dataset(2)
minilm_ndcg_scores = [0.1100458831490401, 0.49276192198755825, 0.15362568563894313, 0.38431135260440824, 0.2899773651602119, 0.17010317160287547, 0.4085359184257957, 0.06625422345438896, 0.0, 0.13698471029831177, 0.3995688713838976, 0.46467597166437496, 0.39366982114002336, 0.4373524791503101, 0.2914893383161489, 0.8582425318466904, 0.496314133361996, 0.5708588662113935, 0.4826073387355251, 0.7799082337019198, 0.25880978898961277, 0.4440973278132557, 0.5648554320973912, 0.31093196555772146, 0.0, 0.23758350840568815, 0.8275534295258489, 0.5912463455842708, 0.6515402749735272, 0.9216017310213245, 0.4511074772466062, 0.0, 0.6203974344166848, 0.0, 0.03312711172719448, 0.6906478832608419, 0.4641183337813128, 0.36642211459480156, 0.9633180389503199, 0.9574284411791895, 0.9681896059005243, 0.8899541168509599, 0.8701249883466594, 0.7818745837832112, 0.6535073651712326, 0.866947989864271, 0.37307760663528433, 0.94497705842548, 0.333391619621485, 0.4225749983705864]

# 55%:
# roberta_large_ndcg_scores_masked_to_openai = calculate_ndcg_scores_sentence_transformer_limited_to_openai('roberta-large')
# roberta_large_ndcg_scores_masked_to_openai = [0.1526174419698506, 0.6410457898283652, 0.5313216166124811, 0.32730659600127, 0.6493956800157978, 0.19200765187283184, 0.20248323207250618, 0.5670429581667767, 0.43483530571065243, 0.15139734109600997, 0.0, 0.1159354809201706, 0.3417690923079354, 0.28510343554781303, 0.10281992268828936, 0.8409421369006526, 0.8066756304994496, 0.7493668840625239, 0.45783805463287947, 0.914856882358379, 0.4200295294463964, 0.6745711375739727, 0.6056113032143303, 0.23992089480238074, 0.1388624438735545, 0.2940945979596687, 0.6994739030813918, 0.58874982024632, 0.42679693286721787, 0.8572048559638626, 0.06625422345438896, 0.0, 0.3360272472182508, 0.06943122193677725, 0.0, 0.6580831550106186, 0.5220328596374271, 0.7206210196320116, 0.9652843890316114, 0.9118499466982855, 0.6633398556941038, 1.0, 0.6717589890736099, 0.8058779736165045, 0.6024171237592955, 0.610206738115618, 0.38541299932114076, 0.8482378089219645, 0.1585070397409811, 0.4142137926815757]
# 100%:
# roberta_large_ndcg_scores = calculate_ndcg_scores_sentence_transformer_full_dataset('roberta-large')
roberta_large_ndcg_scores =[0.1100458831490401, 0.6659485553357843, 0.5670647108745396, 0.47095699092922255, 0.5423640154200349, 0.11101497038372705, 0.21501812929496164, 0.3495957416360196, 0.43128045638816687, 0.06625422345438896, 0.0, 0.031810394099475836, 0.27566553123096654, 0.3787833386759353, 0.06943122193677725, 0.7753366239771086, 0.8300792533881372, 0.7407090992125821, 0.37111125655399296, 0.7849818707050383, 0.3864438606792726, 0.8200685179888283, 0.7459781620647852, 0.3701854920515037, 0.2489083270225946, 0.1794771050858174, 0.715550648768261, 0.8200685179888283, 0.20483424751859086, 0.7721596254947203, 0.07839826897867538, 0.0, 0.21101571757181772, 0.06943122193677725, 0.0, 0.7264127302306636, 0.442017625612159, 0.735488778956926, 0.9305687780632227, 0.9526058178152246, 0.8098106737790872, 1.0, 0.5933607200949346, 0.7338014706205254, 0.6100364306762963, 0.933745776545611, 0.3012042966046398, 1.0, 0.29922683201146266, 0.5164235740297879]

# 55%:
# sgpt_125M_ndcg_scores_masked_to_openai = calculate_ndcg_scores_sentence_transformer_limited_to_openai('SGPT-125M-weightedmean-nli-bitfit')
# sgpt_125M_ndcg_scores_masked_to_openai = [0.03919913448933769, 0.0, 0.4730272350194151, 0.18844415212771548, 0.0, 0.30635490529973475, 0.5328965452000554, 0.4886446891881014, 0.0, 0.4131279133289227, 0.0, 0.0, 0.22883763735188417, 0.0, 0.15840915256850244, 0.13147032541923082, 0.38028902407143617, 0.7803864358037507, 0.05502294157452005, 0.7510916729774054, 0.3125291152215463, 0.513528549750033, 0.1862566260583299, 0.06625422345438896, 0.5215784827120726, 0.4671223597454321, 0.5766882048947065, 0.7702043899253147, 0.4576752747037683, 0.4250887937457822, 0.06362078819895167, 0.042571558820810505, 0.5258863957667049, 0.07839826897867538, 0.1243422521309587, 0.28886316319212685, 0.3347092591719315, 0.4751168795544826, 0.818005721474525, 0.9681896059005243, 0.9668728882728055, 0.8990301655772223, 0.9019353824461352, 0.933745776545611, 0.6020780239162796, 0.0, 0.1388624438735545, 0.1498113059440708, 0.2212088779621395, 0.2200917662980802]
# 100%:

# 55%:
# sgpt_1_3B_ndcg_scores_masked_to_openai = calculate_ndcg_scores_sentence_transformer_limited_to_openai('SGPT-1.3B-weightedmean-nli-bitfit')
# sgpt_1_3B_ndcg_scores_masked_to_openai = [0.042571558820810505, 0.2344065099232998, 0.4854538877975283, 0.0, 0.36966643130673016, 0.4440973278132557, 0.5462574254588145, 0.5033242827953465, 0.0, 0.3295708308392408, 0.0, 0.0, 0.2017507857732402, 0.0, 0.4035015715464804, 0.7971389170873964, 0.5226027621892171, 0.6209886708112745, 0.1017640840433747, 0.8643145546088338, 0.6594803070902429, 0.7381222142123632, 0.46056057476285306, 0.1610425878239397, 0.77523873680463, 0.5017100640839689, 0.7534494445524138, 0.7799082337019198, 0.4193979998881981, 0.39076460427111054, 0.530100330015507, 0.315046288661204, 0.660068415675558, 0.1585070397409811, 0.42010339045270617, 0.28634598975246917, 0.48499195269313716, 0.9363792118010483, 0.9276737537834677, 0.94497705842548, 0.8553475075669357, 0.9305687780632227, 1.0, 0.8087436993719216, 0.4711936318646434, 0.17318663334822615, 0.40957306400643717, 0.8324140018367822, 0.6461194268314838, 0.2200917662980802]
# 100%:


In [ ]:
repr(roberta_large_ndcg_scores)

In [ ]:

print(f"NDCG@10 scores:")
print(f"MiniLM                               {np.average(minilm_ndcg_scores):.2f}")
print(f"roberta-large-v1                     {np.average(roberta_large_ndcg_scores):.2f}")
# print(f"SGPT-125M-weightedmean-nli-bitfit    {np.average(sgpt_125M_ndcg_scores_masked_to_openai):.2f}")
# print(f"SGPT-1.3B-weightedmean-nli-bitfit    {np.average(sgpt_1_3B_ndcg_scores_masked_to_openai):.2f}")
print(f"OpenAI embedding-ada-002_v2          {np.average(openai_ndcg_scores):.2f}")
print(f"calculated on 100% of TREC-COVID dataset")
# print(f"calculated on ~{len(embeddings_df['_id']) / len(corpus_dataset['_id']) * 100 :.0f}% of TREC-COVID dataset")

```
NDCG@10 scores:
MiniLM                               0.43
roberta-large-v1                     0.48
SGPT-125M-weightedmean-nli-bitfit    0.36
SGPT-1.3B-weightedmean-nli-bitfit    0.48
OpenAI embedding-ada-002_v2          0.64
calculated on ~55% of TREC-COVID dataset
```




```
Self reported scores (ndcg@10):

e5-large                            0.78
e5-base                             0.80
e5-small                            0.77
SGPT-5.8B-weightedmean-nli-bitfit   0.55
```
https://docs.google.com/spreadsheets/d/1L8aACyPaXrL8iEelJLGqlMqXKPX2oSP_R10pZoy77Ns/edit#gid=0



### Generating pics for openai only

In [ ]:

plt.rcParams["figure.figsize"] = (14, 20)
PROPS = {
    'boxprops':{'edgecolor':'black', 'zorder': 2},
    'medianprops':{'color':'black'},
    'whiskerprops':{'color':'black'},
    'capprops':{'color':'black'}
}

# corpus_ids_sublist = list(embeddings_df['_id'])
# corpus_ids_subset = set(corpus_ids_sublist)

# experiment_name = 2
# docs_miniLM_embeddings_dict = {_id: embedding for embedding, _id in zip(np.load(f"{drive_root}/{experiment_name}_embeddings_0:None.npy"), corpus_dataset['_id']) 
#                         if _id in corpus_ids_subset}
# docs_miniLM_embeddings_ordered_by_corpus_ids_sublist = \
#     torch.tensor(np.array([docs_miniLM_embeddings_dict[_id] for _id in corpus_ids_sublist]), device=device)
# query_miniLM_embeddings =  torch.tensor(np.load(f"{drive_root}/{experiment_name}_query_embeddings_0:None.npy"), device=device)

# docs_ids = corpus_dataset['_id']
query_ids = queries_dataset['_id']

def generate_dists(_query_ids):

    plt.figure()
    labels = []
    cos_sims = []
    colors = []
    for i, query_id in enumerate([int(_id) for _id in _query_ids]):
        openai_query_embedding = torch.tensor(query_embeddings_df[query_embeddings_df['_id'] == query_id].drop('_id', axis=1).to_numpy(), device=device)
        openai_doc_embeddings = torch.tensor(embeddings_df.drop('_id', axis=1).to_numpy(), device=device)
        openai_cos_sims = cosine_similarity(openai_query_embedding, openai_doc_embeddings).cpu().numpy()
        cos_sims.extend(openai_cos_sims)
        labels.extend([f"query_id = {query_id}; NDCG@10 = {openai_ndcg_scores[query_id - 1]:.2f}" for i in range(len(openai_cos_sims))])
        colors.extend([f"openai" for i in range(len(openai_cos_sims))])

    # print()
    df=pd.DataFrame({'queries': labels, 'cos_sims': cos_sims, 'colors': colors})        
    sns.violinplot(data=df, x='cos_sims', y='queries', inner=None, color='#5FBEF2')
    sns.boxplot(data=df, x='cos_sims', y='queries', saturation=0.5, width=0.2, color='#CCF6FF', **PROPS)
    # plt.legend()   
    plt.title("openai cos_sims distribution")
    plt.grid(axis='x')
    plt.savefig(f"{drive_root}/openai_violin_fig_{_query_ids[0]}-{_query_ids[-1]}.png", dpi=300,  bbox_inches='tight')
    plt.show()
generate_dists(query_ids[0:10])

In [ ]:
for i in range(5):
    generate_dists(query_ids[10*i:10*(i+1)])

In [ ]:
## Generating pics for both
plt.rcParams["figure.figsize"] = (14, 20)
PROPS = {
    'boxprops':{'edgecolor':'black', 'zorder': 2},
    'medianprops':{'color':'black'},
    'whiskerprops':{'color':'black'},
    'capprops':{'color':'black'}
}

corpus_ids_sublist = list(embeddings_df['_id'])
corpus_ids_subset = set(corpus_ids_sublist)

experiment_name = 2
docs_miniLM_embeddings_dict = {_id: embedding for embedding, _id in zip(np.load(f"{drive_root}/{experiment_name}_embeddings_0:None.npy"), corpus_dataset['_id']) 
                        if _id in corpus_ids_subset}
docs_miniLM_embeddings_ordered_by_corpus_ids_sublist = \
    torch.tensor(np.array([docs_miniLM_embeddings_dict[_id] for _id in corpus_ids_sublist]), device=device)
query_miniLM_embeddings =  torch.tensor(np.load(f"{drive_root}/{experiment_name}_query_embeddings_0:None.npy"), device=device)

docs_ids = corpus_dataset['_id']
query_ids = queries_dataset['_id']

def generate_dists(_query_ids):

    plt.figure()
    labels = []
    cos_sims = []
    colors = []
    for i, query_id in enumerate([int(_id) for _id in _query_ids]):
        openai_query_embedding = torch.tensor(query_embeddings_df[query_embeddings_df['_id'] == query_id].drop('_id', axis=1).to_numpy(), device=device)
        openai_doc_embeddings = torch.tensor(embeddings_df.drop('_id', axis=1).to_numpy(), device=device)
        openai_cos_sims = cosine_similarity(openai_query_embedding, openai_doc_embeddings).cpu().numpy()
        cos_sims.extend(openai_cos_sims)
        labels.extend([f"query_id = {query_id}; NDCG@10 = {openai_ndcg_scores[query_id - 1]:.2f}" for i in range(len(openai_cos_sims))])
        colors.extend([f"OpenAI embedding-ada-002_v2" for i in range(len(openai_cos_sims))])

        minilm_cos_sims = cosine_similarity(query_miniLM_embeddings[query_id - 1], docs_miniLM_embeddings_ordered_by_corpus_ids_sublist).cpu().numpy()
        cos_sims.extend(minilm_cos_sims)
        assert len(minilm_cos_sims) == len(openai_cos_sims)
        labels.extend([f"query_id = {query_id}; NDCG@10 = {minilm_ndcg_scores_masked_to_openai[query_id - 1]:.2f}" for i in range(len(minilm_cos_sims))])
        colors.extend([f"all-MiniLM-L6-v2" for i in range(len(minilm_cos_sims))])

    df=pd.DataFrame({'queries': labels, 'cos_sims': cos_sims, 'colors': colors})        
    sns.violinplot(data=df, x='cos_sims', y='queries', hue='colors', inner=None, dodge=False, 
                   palette={"OpenAI embedding-ada-002_v2": "#2B8E46", "all-MiniLM-L6-v2": '#5FBEF2'})
    ax = sns.boxplot(data=df, x='cos_sims', y='queries', hue='colors', saturation=0.5, width=0.2, dodge=False, **PROPS,
                palette={"OpenAI embedding-ada-002_v2": "#D8F9DA", "all-MiniLM-L6-v2": '#CCF6FF'})
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles[0:2], labels=labels[0:2])
    sns.move_legend(ax, "upper left")
    # plt.legend()   
    plt.title("openai cos_sims distribution")
    plt.grid(axis='x')
    plt.savefig(f"{drive_root}/comparison_violin_fig_{_query_ids[0]}-{_query_ids[-1]}.png", dpi=300,  bbox_inches='tight')
    plt.show()
generate_dists(query_ids[0:5])

In [ ]:
for i in range(10):
    generate_dists(query_ids[5*i:5*(i+1)])